In [11]:
from email import header
import requests
import json
from lxpy import copy_headers_dict
import pandas as pd
import os 
from os import listdir
from os.path import isfile, join
# import tkinter as tk
# import tkinter.ttk as ttk
import datetime
from bs4 import BeautifulSoup as bs
import aiohttp
import time
import asyncio
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import sqlite3
import numpy as np
from sqlalchemy import create_engine
from tqdm.notebook import tqdm
import aiohttp
import asyncio
from bs4 import BeautifulSoup as bs
import nest_asyncio

In [3]:
set_headers=copy_headers_dict('''
accept:text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
accept-encoding:gzip, deflate, br, zstd
accept-language:en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7
cache-control:no-cache
pragma:no-cache
priority:u=0, i
sec-ch-ua:"Chromium";v="130", "Microsoft Edge";v="130", "Not?A_Brand";v="99"
sec-ch-ua-mobile:?0
sec-ch-ua-platform:"macOS"
sec-fetch-dest:document
sec-fetch-mode:navigate
sec-fetch-site:none
sec-fetch-user:?1
upgrade-insecure-requests:1
user-agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0
''')
url="https://accessnc.nccommerce.com/BusinessSearch/Company/list"
trash='''file:///'''
counties = [
    "Alexander", "Alleghany", "Ashe", "Avery", "Buncombe", "Burke",
    "Caldwell", "Catawba", "Clay", "Cleveland", "Gaston", "Haywood",
    "Henderson", "Jackson", "Lincoln", "Macon", "Madison", "McDowell",
    "Mecklenburg", "Mitchell", "Polk", "Rutherford", "Swain",
    "Transylvania", "Watauga", "Wilkes", "Yancey"
]


## Example Access

In [4]:
data = {
    "regionCategory": "County",  # Example: "County" or "Region"
    "region": "Alexander",  # Example: "Wake" for Wake County
    "IndustryGroup2": "1124",  # NAICS 4-digit code, e.g., "1124" for Sheep and Goat Farming
    # "CompanyName": "",  # Optional: Add a company name or keyword
    # "stype": "c",  # Match type for CompanyName: 'c' (contains) or 's' (starts with)
    # "STFIPS": "37",  # State FIPS code for North Carolina
}

# Send POST request
response = requests.post(url, data=data)

# Check the response
if response.status_code == 200:
    print("Search successful!")
    print(response.text)  # or parse the response as needed
else:
    print(f"Search failed with status code {response.status_code}")


Search successful!

<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Business Search</title>
    <link href="/BusinessSearch/Content/css?v=MDbdFKJHBa_ctS5x4He1bMV0_RjRq8jpcIAvPpKiN6U1" rel="stylesheet"/>

    <script src="/BusinessSearch/bundles/modernizr?v=wBEWDufH_8Md-Pbioxomt90vm6tJN2Pyy9u9zHtWsPo1"></script>


</head>
<body>
    <div class="navbar navbar-inverse navbar-fixed-top">
        <a class="navbar-brand" href="https://d4.nccommerce.com">D4</a>
        
    </div>
    <div class="container body-content">
        


<link href="/BusinessSearch/Content/PagedList.css" rel="stylesheet" type="text/css" />



<h3>Business Search Results</h3>
<p>
    Business Search Results  657977 matched records   
     
</p>
<table class="table">
    <tr>
        <th>
            <a href="/BusinessSearch/Company/sort?sortOrder=company">Company</a>
        </th>
        <th>
            <a href="/Busin

In [5]:
html_parse=bs(response.text,'html.parser')
for link in html_parse.find_all("a", string=lambda text: text == "Detail"):
    print(link)

<a href="/BusinessSearch/Company/Detail/762283043">Detail</a>
<a href="/BusinessSearch/Company/Detail/734095575">Detail</a>
<a href="/BusinessSearch/Company/Detail/745908587">Detail</a>
<a href="/BusinessSearch/Company/Detail/723354960">Detail</a>
<a href="/BusinessSearch/Company/Detail/740424324">Detail</a>
<a href="/BusinessSearch/Company/Detail/401678984">Detail</a>
<a href="/BusinessSearch/Company/Detail/220228084">Detail</a>
<a href="/BusinessSearch/Company/Detail/705185549">Detail</a>
<a href="/BusinessSearch/Company/Detail/725665057">Detail</a>
<a href="/BusinessSearch/Company/Detail/717072561">Detail</a>
<a href="/BusinessSearch/Company/Detail/435029225">Detail</a>
<a href="/BusinessSearch/Company/Detail/742698417">Detail</a>
<a href="/BusinessSearch/Company/Detail/722866552">Detail</a>
<a href="/BusinessSearch/Company/Detail/742138490">Detail</a>
<a href="/BusinessSearch/Company/Detail/722799086">Detail</a>
<a href="/BusinessSearch/Company/Detail/747310914">Detail</a>
<a href=

## All Links

In [13]:
# Read NAICS codes from the file
with open("qnaics_codes.txt", 'r') as f:
    codes = f.readlines()

# Clean and prepare data
codes = list(map(lambda x: x.strip(), codes))
datalist = []
for county in counties:
    for code in codes:
        datalist.append({
            "regionCategory": "County",  # Example: "County" or "Region"
            "region": county,  # Example: "Wake" for Wake County
            "IndustryGroup2": code,  # NAICS 4-digit code, e.g., "1124" for Sheep and Goat Farming
            # "CompanyName": "",  # Optional: Add a company name or keyword
            # "stype": "c",  # Match type for CompanyName: 'c' (contains) or 's' (starts with)
            # "STFIPS": "37",  # State FIPS code for North Carolina
        })

# URL for form submission
url = "https://accessnc.nccommerce.com/BusinessSearch/Company/list"


In [19]:

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()


# Asynchronous function to fetch and process data
async def fetch_data(session, data):
    try:
        await asyncio.sleep(0.05)
        async with session.post(url, data=data) as response:
            if response.status == 200:
                html_parse = bs(await response.text(), 'html.parser')
                
                # Extract detail links
                links = []
                for link in html_parse.find_all("a", string=lambda text: text == "Detail"):
                    local_link = link.get("href")
                    links.append(local_link)
                    with open("all_links_counties.txt", "a") as f:
                        f.write(f"{local_link}\n")
                
                return links
            else:
                print(f"Failed for data: {data['region']} {data['IndustryGroup2']} with status {response.status}")
    except Exception as e:
        print(f"Error processing data: {data['region']} {data['IndustryGroup2']} - {e}")

# Main asynchronous function to handle all requests with progress bar
async def process_datalist(datalist):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_data(session, data) for data in datalist]
        results = []
        for f in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Processing NAICS codes"):
            result = await f
            results.append(result)
        return results

# Run the asynchronous main function in Jupyter
async def main():
    results = await process_datalist(datalist)
    print("All data processed")

# Start the event loop in Jupyter
await main()

Processing NAICS codes:   0%|          | 0/16794 [00:00<?, ?it/s]

Error processing data: Cleveland 1111 - 
Error processing data: McDowell 1129 - 
Error processing data: Caldwell 4851 - 
Error processing data: Ashe 7113 - 
Error processing data: Alexander 4871 - 
Error processing data: Alleghany 5221 - 
Error processing data: Watauga 4248 - 
Error processing data: Swain 5173 - 
Error processing data: Rutherford 2122 - 
Error processing data: McDowell 1125 - 
Error processing data: Caldwell 4842 - 
Error processing data: Ashe 7112 - 
Error processing data: Watauga 4247 - 
Error processing data: Swain 5152 - 
Error processing data: Swain 4431 - 
Error processing data: Rutherford 2121 - 
Error processing data: McDowell 2211 - 
Error processing data: Caldwell 4841 - 
Error processing data: Ashe 7111 - 
Error processing data: Alleghany 5241 - 
Error processing data: Watauga 4246 - 
Error processing data: Swain 5151 - 
Error processing data: Rutherford 2111 - 
Error processing data: McDowell 2131 - 
Error processing data: Caldwell 4832 - 
Error processing 

In [20]:
all_links = []
base_url = "https://accessnc.nccommerce.com"
with open("all_links_counties.txt","r") as f:
    all_links = f.readlines()
all_links_full = list(set(list(map(lambda x: base_url + x, all_links))))
with open("all_links_counties_full.txt","w") as f:
    f.writelines(all_links_full)